In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2

# Converting Segmentation Masks to Bounding Boxes in YOLO Format

Input : Directory where the masks are in .png format  
Output : Directory where YOLO Annotations are in .txt format

In [ ]:
def convert_png_to_yolo(input_filepath, output_filepath, class_id = 0):

  img = cv2.imread(input_filepath, cv2.IMREAD_GRAYSCALE)
  contours , _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  output_dir = os.path.dirname(output_filepath)
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  with open(output_filepath, 'w') as f:
    for contour in contours:
      x,y,w,h = cv2.boundingRect(contour)

      #Normalizing Coordinates in the range [0,1]

      img_width, img_height = img.shape[1], img.shape[0]
      x_center = (x + w/2)/img_width
      y_center = (y + h/2)/img_height
      width = w / img_width
      height = h / img_height

      f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


def convert_labels(input_dir, output_dir):
  for eachfile in os.listdir(input_dir):
    if eachfile.endswith('.png'):
      png_filepath = os.path.join(input_dir, eachfile)
      txt_filepath = os.path.join(output_dir, eachfile.replace('.png','.txt'))

      convert_png_to_yolo(png_filepath, txt_filepath)


In [ ]:
train_mask_dir = "/content/drive/MyDrive/Nepal landslide segmentation dataset/Training/label"
test_mask_dir = "/content/drive/MyDrive/Nepal landslide segmentation dataset/Test/mask"
valid_mask_dir = "/content/drive/MyDrive/Nepal landslide segmentation dataset/Validation/masks"

train_box_dir = "/content/drive/MyDrive/dataset/labels/train_new"
test_box_dir = "/content/drive/MyDrive/dataset/labels/test_new"
valid_box_dir = "/content/drive/MyDrive/dataset/labels/val_new"

In [ ]:
convert_labels(train_mask_dir, train_box_dir)
convert_labels(test_mask_dir, test_box_dir)
convert_labels(valid_mask_dir, valid_box_dir)

# Exploring the .tiff data

In [ ]:
!pip install rasterio

import rasterio as r
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show

tiff_file = '/content/drive/MyDrive/major_proj/dataset_original_tiff/images/train/b_COMP_0_1.tiff'

with r.open(tiff_file) as src:
  print(f"Metadata : \n {src.meta}")
  tiff_data = src.read()

print("Number of bands:", src.count)         # Number of bands
print("Image size (width x height):", src.width, "x", src.height)
print("Coordinate Reference System (CRS):", src.crs)  # Spatial reference
print("Bounds:", src.bounds)                # Geospatial bounds




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 44.5 MB/s eta 0:00:00
Metadata : 
 {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 179, 'height': 158, 'count': 3, 'crs': None, 'transform': Affine(1.0, 0.0, 0.0,
       0.0, 1.0, 0.0)}
Number of bands: 3
Image size (width x height): 179 x 158
Coordinate Reference System (CRS): None
Bounds: BoundingBox(left=0.0, bottom=158.0, right=179.0, top=0.0)


/usr/local/lib/python3.11/dist-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


## Converting .tiff format to .jpg


In [ ]:
from PIL import Image
import os

In [ ]:

image_dir = '/content/drive/MyDrive/dataset/images/train'
output_dir = '/content/drive/MyDrive/dataset/images/train_jpg'

for filename in os.listdir(image_dir):
  if filename.endswith('.tiff'):
    image_path = os.path.join(image_dir,filename)
    output_path = os.path.join(output_dir, filename.replace('.tiff','.jpg'))
    image = Image.open(image_path)
    image.save(output_path, 'JPEG')


In [ ]:
image_dir = '/content/drive/MyDrive/dataset/images/test'
output_dir = '/content/drive/MyDrive/dataset/images/test_jpg'

for filename in os.listdir(image_dir):
  if filename.endswith('.tiff'):
    image_path = os.path.join(image_dir,filename)
    output_path = os.path.join(output_dir, filename.replace('.tiff','.jpg'))
    image = Image.open(image_path)
    image.save(output_path, 'JPEG')

In [ ]:
image_dir = '/content/drive/MyDrive/dataset/images/val'
output_dir = '/content/drive/MyDrive/dataset/images/val_jpg'

for filename in os.listdir(image_dir):
  if filename.endswith('.tiff'):
    image_path = os.path.join(image_dir,filename)
    output_path = os.path.join(output_dir, filename.replace('.tiff','.jpg'))
    image = Image.open(image_path)
    image.save(output_path, 'JPEG')

# Making sure the source and labels have the same name

In [ ]:

import os
import shutil

source_directory = "/content/drive/MyDrive/dataset/labels/train"

target_directory = "/content/drive/MyDrive/dataset/labels/train_new"

os.makedirs(target_directory, exist_ok=True)

for filename in os.listdir(source_directory):
    old_file_path = os.path.join(source_directory, filename)

    if os.path.isfile(old_file_path) and "mask" in filename:
        new_filename = filename.replace("_mask", "")
        new_filename = f"b_{new_filename}"
        new_file_path = os.path.join(target_directory, new_filename)

        shutil.copy(old_file_path, new_file_path)
        print(f"Copied and renamed: {filename} -> {new_filename}")


Copied and renamed: COMP_1_1_mask.txt -> b_COMP_1_1.txt
Copied and renamed: COMP_2_3_mask.txt -> b_COMP_2_3.txt
Copied and renamed: COMP_2_4_mask.txt -> b_COMP_2_4.txt
Copied and renamed: COMP_5_1_mask.txt -> b_COMP_5_1.txt
Copied and renamed: COMP_5_2_mask.txt -> b_COMP_5_2.txt
Copied and renamed: COMP_5_4_mask.txt -> b_COMP_5_4.txt
Copied and renamed: COMP_5_7_mask.txt -> b_COMP_5_7.txt
Copied and renamed: COMP_5_10_mask.txt -> b_COMP_5_10.txt
Copied and renamed: COMP_5_12_mask.txt -> b_COMP_5_12.txt
Copied and renamed: COMP_5_15_mask.txt -> b_COMP_5_15.txt
Copied and renamed: COMP_5_16_mask.txt -> b_COMP_5_16.txt
Copied and renamed: COMP_5_18_mask.txt -> b_COMP_5_18.txt
Copied and renamed: COMP_5_20_mask.txt -> b_COMP_5_20.txt
Copied and renamed: COMP_5_21_mask.txt -> b_COMP_5_21.txt
Copied and renamed: COMP_5_23_mask.txt -> b_COMP_5_23.txt
Copied and renamed: COMP_5_25_mask.txt -> b_COMP_5_25.txt
Copied and renamed: COMP_5_26_mask.txt -> b_COMP_5_26.txt
Copied and renamed: COMP_5_3

# Visualizing the Labels


In [ ]:
#Visualizing the labels by overlaying the bounding boxes over the images

import cv2
from PIL import Image
import numpy as np
from google.colab.patches import cv2_imshow

def visualize_bb(img_path,annotation_path,save_path):
  img = Image.open(img_path)
  img_np = np.array(img)

  img_height, img_width = img_np.shape[:2]

  with open(annotation_path , 'r') as f:
    annotation = f.readlines()

    for line in annotation:
      x,y,w,h = map(float, line.split()[1:5])

      x = x * img_width
      y = y * img_height
      w = w * img_width
      h = h * img_height

      x_min = int(x-w/2)
      y_min = int(y-h/2)
      x_max = int(x+w/2)
      y_max = int(y+h/2)


      cv2.rectangle(img_np,
                    (x_min,y_min),
                    (x_max,y_max),
                    (255,0,0),
                    1)
      cv2.imwrite(save_path, img_np)




In [ ]:
# The labels look fairly okay. Lets do this for all the images and masks
import os

dest = '/content/drive/MyDrive/dataset_with_boxes_visualized/val'
train_img_src = '/content/drive/MyDrive/major_proj/dataset_original_tiff/images/val'
train_label_src = '/content/drive/MyDrive/major_proj/dataset_original_tiff/labels/val'

if not os.path.exists(dest):
  os.makedirs(dest)

for eachfile in os.listdir(train_label_src):
  file_path = os.path.join(train_label_src, eachfile)
  img_path = os.path.join(train_img_src, eachfile.replace('txt','tiff'))
  dest_path = os.path.join(dest,eachfile.replace('txt','png'))

  visualize_bb(img_path,file_path,dest_path)


# Training the Data


# Cloning the YOLOv9 Repository and Downloading the pretrained weights

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git
%cd yolov9
!pip install -r requirements.txt

In [ ]:
#Downloading the YOLOv9 and GELAN Pretrained Weights

!wget -P  https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt
!wget -P https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
%cd '/content/drive/MyDrive/major_proj/yolov9'

/content/drive/MyDrive/major_proj/yolov9


# Training

In [ ]:

!python train.py \
--batch 4 --epochs 500 --img 416 --device 0 --min-items 0 --close-mosaic 15 \
--data '/content/drive/MyDrive/major_proj/dataset_original_tiff/data.yaml' \
--weights '/content/drive/MyDrive/major_proj/yolov9/weights/gelan-e.pt' \
--cfg models/detect/gelan-e.yaml \
--hyp hyp.scratch-high.yaml

2025-01-22 04:28:20.340938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-22 04:28:20.379841: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-22 04:28:20.391650: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 04:28:20.425790: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-22 04:28:22.641835: W tensorflow/compiler/tf2

# Inference

In [ ]:
!python detect.py \
--img 416 --conf 0.1 --device 0 \
--weights /content/drive/MyDrive/major_proj/yolov9/runs/train/exp28/weights/best.pt \
--source /content/drive/MyDrive/major_proj/dataset_original_tiff/images/test \
--line-thickness 1 \
--hide-labels

import glob
from IPython.display import Image, display

for image_path in glob.glob(f'/content/drive/MyDrive/yolo/yolov9/runs/detect/exp8/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

detect: weights=['/content/drive/MyDrive/major_proj/yolov9/runs/train/exp28/weights/best.pt'], source=/content/drive/MyDrive/major_proj/dataset_original_tiff/images/test, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=True, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

/content/drive/MyDrive/major_proj/yolov9/models/experimental.py:243: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch

2025-01-21 15:50:24.010205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-21 15:50:24.030354: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-21 15:50:24.036468: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 15:50:24.052146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-21 15:50:25.312628: W tensorflow/compiler/tf2